# With 3 features

In [11]:
 #libraries
import pandas as pd # for data analysis
import csv #load the data
import numpy as np #for array computation

from sklearn.ensemble import RandomForestClassifier 
# Create a random forest Classifier. By convention, clf means 'Classifier'
from sklearn.metrics import accuracy_score # to get some statistical reports 
from sklearn.svm import SVC

In [2]:
 #load the dataset
players = pd.read_csv('players.csv', index_col=0)
test_labels = pd.read_csv('test_labels.csv', index_col=0) 
test_players = pd.read_csv('test_player.csv', index_col=0)
match = pd.read_csv('match.csv', index_col=0) 

In [133]:
 match.head() # 'radiant_win' column required as target variable

,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_dire,barracks_status_radiant,first_blood_time,game_mode,radiant_win,negative_votes,positive_votes,cluster
match_id,,,,,,,,,,,,
0,1446750112,2375,1982,4,3,63,1,22,True,0,1,155
1,1446753078,2582,0,1846,63,0,221,22,False,0,2,154
2,1446764586,2716,256,1972,63,48,190,22,False,0,0,132
3,1446765723,3085,4,1924,51,3,40,22,False,0,0,191
4,1446796385,1887,2047,0,0,63,58,22,True,0,0,156


In [134]:
players.head() # for features

,account_id,hero_id,player_slot,gold,gold_spent,gold_per_min,xp_per_min,kills,deaths,assists,...,unit_order_glyph,unit_order_eject_item_from_stash,unit_order_cast_rune,unit_order_ping_ability,unit_order_move_to_direction,unit_order_patrol,unit_order_vector_target_position,unit_order_radar,unit_order_set_item_combine_lock,unit_order_continue
match_id,,,,,,,,,,,,,,,,,,,,,
0,0,86,0,3261,10960,347,362,9,3,18,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
0,1,51,1,2954,17760,494,659,13,3,18,...,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN
0,0,83,2,110,12195,350,385,0,4,15,...,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN
0,2,11,3,1179,22505,599,605,8,4,19,...,1.0,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN
0,3,67,4,3307,23825,613,762,20,3,17,...,3.0,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = pd.DataFrame(players, columns = ['account_id','hero_id','player_slot'])
#feature selection which are in our test file

In [136]:
df.head()

,account_id,hero_id,player_slot
match_id,,,
0,0,86,0
0,1,51,1
0,0,83,2
0,2,11,3
0,3,67,4


In [4]:
ab = pd.DataFrame(match, columns= ['radiant_win']) # label to predict

In [5]:
 X= df.join(ab) # Add a new column with the radiant_win, this is what we are going to try to predict

In [141]:
print X.head()

          account_id  hero_id  player_slot  radiant_win
match_id                                               
0                  0       86            0         True
0                  1       51            1         True
0                  0       83            2         True
0                  2       11            3         True
0                  3       67            4         True


In [6]:
X['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
# Create a new column that for each row, generates a random number between 0 and 1, and
# if that value is less than or equal to .75, then sets the value of that cell as True
# and false otherwise. This is a quick way of randomly assigning some rows to
#be used as the training data and some as the validation data.



In [147]:
X.head()

,account_id,hero_id,player_slot,radiant_win,is_train
match_id,,,,,
0,0,86,0,True,False
0,1,51,1,True,False
0,0,83,2,True,True
0,2,11,3,True,True
0,3,67,4,True,True


In [7]:
# Create two new dataframes, one with the training rows, one with the test rows
train, test = X[X['is_train']==True], X[X['is_train']==False]

In [155]:
# Show the number of observations for the validation and training dataframes
print('Number of observations in the training data:', len(train))
print('Number of observations in the validation data:',len(test))

('Number of observations in the training data:', 375373)
('Number of observations in the validation data:', 124627)


In [8]:
# Create a list of the feature column's names
features = X.columns[:3]

In [157]:
features

Index([u'account_id', u'hero_id', u'player_slot'], dtype='object')

In [9]:
# train['radiant_win'] and test['radiant_win'] contains the result in true and false form. Before we can use it,
# we need to convert results into a digit. So, in this case there
# are two results, which have been coded as 0 as loss and 1 as win.
y = pd.factorize(train['radiant_win'])[0]
y_val = pd.factorize(test['radiant_win'])[0]
# View target
print y
print y_val

[0 0 0 ..., 1 1 1]
[0 0 0 ..., 0 1 1]


In [12]:

# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = RandomForestClassifier( random_state=0,n_estimators=100)

# Train the Classifier to take the training features and learn how they relate
# to the training y 
clf.fit(train[features], y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [160]:
# Apply the Classifier we trained to the validation data (which, remember, it has never seen before)
clf.predict(test[features])

array([1, 0, 0, ..., 0, 0, 1])

In [161]:
# View the predicted probabilities of the first 10 observations
clf.predict_proba(test[features])[0:10]

array([[ 0.41933371,  0.58066629],
       [ 0.60642134,  0.39357866],
       [ 0.53335414,  0.46664586],
       [ 0.74906858,  0.25093142],
       [ 0.44658843,  0.55341157],
       [ 0.51657623,  0.48342377],
       [ 0.47908019,  0.52091981],
       [ 0.72287819,  0.27712181],
       [ 0.5170508 ,  0.4829492 ],
       [ 0.4711331 ,  0.5288669 ]])

In [163]:
test['radiant_win'].head() ## View the ACTUAL outcome for the first five observations

match_id
0     True
0     True
0     True
1    False
1    False
Name: radiant_win, dtype: bool

In [164]:
preds = match.radiant_win[clf.predict(test[features])]

In [165]:
preds[0:5] # View the PREDICTED outcome for the first five observations

match_id
1    False
0     True
0     True
0     True
1    False
Name: radiant_win, dtype: bool

In [166]:
# View a list of the features and their importance scores
list(zip(train[features], clf.feature_importances_))

[('account_id', 0.88851883680723676),
 ('hero_id', 0.10317784421938155),
 ('player_slot', 0.0083033189733817108)]

In [167]:
 from sklearn.metrics import accuracy_score
predicted = clf.predict(test[features])
accuracy = accuracy_score(y_val, predicted)

In [168]:
print accuracy #on validation set

0.512601603184


In [ ]:
#we used Soft SVM with changing parameter to see the accuracy

In [127]:
 from sklearn.svm import SVC
clf = SVC()
clf.fit(train[features], y) 
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
print('accuracy', clf.score(test[features], y_val), '\n')


('accuracy', 0.52117274243887646, '\n')


In [128]:
  from sklearn.svm import SVC
clf = SVC()
clf.fit(train[features], y) 
SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
print('accuracy', clf.score(test[features], y_val), '\n')


('accuracy', 0.52117274243887646, '\n')


In [ ]:
 #lets calculate the accuracy on given test data
#model selection: after computing SVM with kernels and random forest algorithm
#decided to select random forest algorithm as less computational time it takes and better accuracy on varying parameter

In [13]:
ts = pd.DataFrame(test_players) #store the test data in dataframe
tl = pd.DataFrame(test_labels)

In [14]:
test_set = ts.join(tl)
test_set.head()


,account_id,hero_id,player_slot,radiant_win
match_id,,,,
50000,117784,96,0,0
50000,158361,84,1,0
50000,158362,46,2,0
50000,137970,85,3,0
50000,1090,39,4,0


In [15]:
 y_label = pd.factorize(test_set['radiant_win'])[0] #target variable from test data

In [16]:
from sklearn.metrics import accuracy_score
predicted = clf.predict(ts)
accuracy = accuracy_score(y_label, predicted)
print clf.score(ts,y_label) #accuracy on test data given

0.488596


In [ ]:
#Thank you.